In [ ]:
import pandas as pd
import datetime as dt
from controller.koreainvestment_controller import get_ki_broker
from controller.koreainvestment_controller import ORDER_CONTROLLER

In [ ]:
CFG = {
    "buying_stockcodes": ["000320", "016880"],
}

In [ ]:
ki_broker = get_ki_broker()

In [ ]:
order_controller = ORDER_CONTROLLER(ki_broker)

order_preprocessor = order_controller.order_preprocessor()
order_requester = order_controller.order_requester()

In [ ]:
buying_df = order_preprocessor.get_buying_df(CFG['buying_stockcodes'])

In [ ]:
# buying_df 생성
buying_df = order_preprocessor.get_buying_df(CFG['buying_stockcodes'])

# buying_df 시장가 매수
order_resp_dict = dict()

for idx, row in buying_df.iterrows():
    stockcode = row["StockCode"]
    quantity = row["Quantity"]
    order_resp = order_requester.request_market_buy_order(stockcode=stockcode, quantity=quantity)
    order_resp_dict[stockcode] = order_resp

In [ ]:
open_order_resv = ki_broker.fetch_open_order_resv()

In [ ]:
open_order_resv_df = pd.DataFrame(open_order_resv["output"])
open_order_resv_df.loc[:, ["pdno", "kor_item_shtn_name", "ord_rsvn_qty", "ord_rsvn_unpr"]]

In [ ]:
position_df = order_preprocessor.get_position_df()

In [ ]:
"""
https://blog.naver.com/kyoyo21/223170296790
"""
import numpy as np


def call_price_maker(price):
    def call_price_calc(price, n):
        return np.around(price / n) * n

    if price < 2000:
        call_price = call_price_calc(price, 1)
    elif price < 5000:
        call_price = call_price_calc(price, 5)
    elif price < 20000:
        call_price = call_price_calc(price, 10)
    elif price < 50000:
        call_price = call_price_calc(price, 50)
    elif price < 200000:
        call_price = call_price_calc(price, 100)
    elif price < 500000:
        call_price = call_price_calc(price, 500)
    else:
        call_price = call_price_calc(price, 1000)
    return int(call_price)

In [ ]:
position_df["ResvUpperPrice"] = position_df["BuyingPrice"].apply(lambda x: call_price_maker(x * 1.06))
position_df["ResvLowerPrice"] = position_df["BuyingPrice"].apply(lambda x: call_price_maker(x * 0.98))

In [ ]:
order_resv_resp_dict = dict()

for idx, row in position_df.iterrows():
    StockCode = row["StockCode"]
    Quantity = row["Quantity"]
    ResvUpperPrice = row["ResvUpperPrice"]
    ResvLowerPrice = row["ResvLowerPrice"]
    
    order_resv_lower_resp = order_requester.request_limit_sell_order_resv(
        stockcode=StockCode,
        price=ResvLowerPrice,
        quantity=Quantity,
        resv_date=(dt.date.today() + dt.timedelta(days=10)),
    )
    
    order_resv_upper_resp = order_requester.request_limit_sell_order_resv(
        stockcode=StockCode,
        price=ResvUpperPrice,
        quantity=Quantity,
        resv_date=(dt.date.today() + dt.timedelta(days=10)),
    )

    order_resv_resp_dict[StockCode] = (order_resv_lower_resp, order_resv_upper_resp)

In [ ]:
order_requester.request_limit_sell_order_resv(
    
)

In [ ]:
position_df

In [ ]:
position_df

In [ ]:
params = {
    "CTX_AREA_FK100": "",
    "CTX_AREA_NK100": "",
    "INQR_DVSN_1": "0",
    "INQR_DVSN_2": "0",
}
order_resp = ki_broker.fetch_open_order()

In [ ]:
pd.DataFrame(order_resp["output"])

In [ ]:
class KI_ORDER_CONTROLLER:
    def __init__(self, broker) -> None:
        pass